In [22]:
# day 14 - part 1
# define a function that processes the input
def cleanInput(inputFilename):
  with open(inputFilename, 'r') as file:
    inputData = file.read()

  # general cleaning
  inputData = inputData.strip() # OMG ALWAYS REMEMBER TO STRIP THE INPUT FIRST!!! (maybe i should define a custom load function here)
  inputData = inputData.split('\n')
  # inputData = inputData.split('\n\n')
  # inputData = list(filter(None, inputData)) # i think you only need this because i forgot that you have to strip the inputData first? lmao

  # problem-specific cleaning
  # inputData = tuple(tuple(row) for row in inputData) # to use functools.cache (no this is wrong somehow lmao)
  inputData = [list(row) for row in inputData]
  return inputData

inputFilename = 'advent-of-code-day14-input'
# inputFilename = 'advent-of-code-day14-testcase'
platform = cleanInput(inputFilename)
# platform

In [23]:
# i think it's much easier if it's tranposed
def transpose(matrix):
  return [list(col) for col in zip(*matrix)]

# tiltNorth is really tiltWest when it's all tranposed
def tiltNorth(platform):
  platform = transpose(platform) # turn it into a tiltWest problem, so we can process it in a per row basis
  # the core algo (tiltWest)
  for i, row in enumerate(platform):
    # i think you use two pointers here, sliding window?
    leftPointer, rightPointer = 0, 0
    while leftPointer < len(row) and rightPointer < len(row):
      # OH MY GOD THIS WORKS LET'S GOOOOOO
      if platform[i][leftPointer] != '.': # if leftPointer hasn't found a place to rest yet, just move forward
        leftPointer += 1
        continue
      if leftPointer >= rightPointer: # then after that is satisfied, rightPointer must be larger than leftPointer
        rightPointer = leftPointer + 1
        continue
      if platform[i][rightPointer] == '.': # if rightPointer founds a '.', just move on
        rightPointer += 1
        continue
      if platform[i][rightPointer] == 'O': # if it founds a movable rock, switch places
        platform[i][leftPointer] = 'O'
        platform[i][rightPointer] = '.'
        leftPointer += 1
        rightPointer += 1
        continue
      if platform[i][rightPointer] == '#': # if it founds an immovable rock, update leftPointer
        leftPointer = rightPointer
        continue

  platform = transpose(platform)
  return platform

def countWeightNorth(platform):
  total = 0
  for i, row in enumerate(platform):
    total += sum(1 for el in row if el == 'O') * (len(row) - i)
  return total

# tiltNorth(platform)
countWeightNorth(tiltNorth(platform))
# 136, testcase right
# 105461, right!! after 6 hours of half assing it and 15 minutes of full assing it

105461

In [102]:
# day 14 - part 2
# i think we should refactor this to use tiltWest all the time
def rotateCCW90(matrix):
  return [list(col) for col in zip(*matrix)][::-1]

def tiltWest(platform):
    for i, row in enumerate(platform):
    # i think you use two pointers here, sliding window?
      leftPointer, rightPointer = 0, 0
      while leftPointer < len(row) and rightPointer < len(row):
        # OH MY GOD THIS WORKS LET'S GOOOOOO
        if platform[i][leftPointer] != '.': # if leftPointer hasn't found a place to rest yet, just move forward
          leftPointer += 1
          continue
        if leftPointer >= rightPointer: # then after that is satisfied, rightPointer must be larger than leftPointer
          rightPointer = leftPointer + 1
          continue
        if platform[i][rightPointer] == '.': # if rightPointer founds a '.', just move on
          rightPointer += 1
          continue
        if platform[i][rightPointer] == 'O': # if it founds a movable rock, switch places
          platform[i][leftPointer] = 'O'
          platform[i][rightPointer] = '.'
          leftPointer += 1
          rightPointer += 1
          continue
        if platform[i][rightPointer] == '#': # if it founds an immovable rock, update leftPointer
          leftPointer = rightPointer
          continue
    return platform

def tiltNorth(platform):
  platform = rotateCCW90(platform)
  platform = tiltWest(platform)
  platform = rotateCCW90(rotateCCW90(rotateCCW90(platform))) # lmao i can make a rotateCW90 function but this is just too funny
  return platform

def tiltEast(platform):
  platform = rotateCCW90(rotateCCW90(platform))
  platform = tiltWest(platform)
  platform = rotateCCW90(rotateCCW90(platform))
  return platform

def tiltSouth(platform):
  platform = rotateCCW90(rotateCCW90(rotateCCW90(platform)))
  platform = tiltWest(platform)
  platform = rotateCCW90(platform)
  return platform

# rudimentary cycle finding algo
def findCycle(seq, minCycleLen = 200): # (NOTE) MINLEN IS A HACK HERE
  for start in range(len(seq)):
    for end in range(start + minCycleLen, len(seq)): # we have a minLen here
      if seq[start:end] == seq[end: end + (end - start)]:
        return (start, end, end - start) # OMG THE BUG IS HERE, LEN IS ALWAYS end - start WHEN DEALING WITH LISTS!!
  return None


def totalTiltCycles(platform, multiplier, minCycleLen = 200): # add a minLen hack
  # finding the cycle
  weightCycle = []
  cycleStart = None
  for i in range(multiplier): # the cycle
    platform = tiltEast(tiltSouth(tiltWest(tiltNorth(platform))))
    weight = countWeightNorth(platform)
    if findCycle(weightCycle, minCycleLen): # find cycle, with a minLen
      cycleStart, cycleEnd, cycleLength = findCycle(weightCycle, minCycleLen)
      break
    weightCycle.append(weight)

  # after broken
  if cycleStart is not None:
    print(cycleStart, cycleEnd, cycleLength) # just for debugging
    # we want to know what the value'd be at index = multiplier - 1 (at the end)
    _, remainder = divmod(((multiplier - 1) - cycleStart), cycleLength)
    return weightCycle[cycleStart + remainder] # return the one at remainder # ANOTHER BUG IT SHOULD'VE BEEN cycleStart PLUS(!!!) REMAINDER!!
  return weightCycle[multiplier - 1]

# countWeightNorth is still the same

# solution
# inputFilename = 'advent-of-code-day14-input'
inputFilename = 'advent-of-code-day14-testcase'
multiplier = 10 ** 9
minCycleLen = 300
platform = cleanInput(inputFilename)
totalTiltCycles(platform, multiplier, minCycleLen)
# 64, right for the testcase
# 96657, bismillah ... WRONG (MINLEN = 4) (too low)
# 99872 (MINLEN = 100)
# 102164 (MINLEN = 200) hmm this approach is not stable lmao
# 102827 (MINLEN = 300)

# there might be bug here length is always end - start, never end - start + 1
# 102831 (MINLEN = 300)
# 102831 (MINLEN = 200)
# 102831 (MINLEN = 400) WRONG TOO HIGH!!!!

# third try
# 102829 (minCycleLen = 200)
# 102829 (minCycleLen = 300)
# 102829 okay let's try this.. bismillah RIGHT!!!
# oh my god this took me 4 HOURS
# 64, right for the test case too
# wow this was brutal oh my god

2 303 301


64

# scratchpad

In [ ]:
## (NOTE) THE CYCLE FINDING ALGO DOESN'T WORK HERE WHEN YOU GOT SEQUENCES LIKE THIS
# 87, 69, 69, 69, 65, 64, 65, 63, 68, 69, 69, 65, 64, 65, 63, 68, 69, 69, 65, 64, 65, 63, 68, 69, 69, 65, 64, 65, 63

# day 14 - part 2
# i think we should refactor this to use tiltWest all the time
def rotateCCW90(matrix):
  return [list(col) for col in zip(*matrix)][::-1]

def tiltWest(platform):
    for i, row in enumerate(platform):
    # i think you use two pointers here, sliding window?
      leftPointer, rightPointer = 0, 0
      while leftPointer < len(row) and rightPointer < len(row):
        # OH MY GOD THIS WORKS LET'S GOOOOOO
        if platform[i][leftPointer] != '.': # if leftPointer hasn't found a place to rest yet, just move forward
          leftPointer += 1
          continue
        if leftPointer >= rightPointer: # then after that is satisfied, rightPointer must be larger than leftPointer
          rightPointer = leftPointer + 1
          continue
        if platform[i][rightPointer] == '.': # if rightPointer founds a '.', just move on
          rightPointer += 1
          continue
        if platform[i][rightPointer] == 'O': # if it founds a movable rock, switch places
          platform[i][leftPointer] = 'O'
          platform[i][rightPointer] = '.'
          leftPointer += 1
          rightPointer += 1
          continue
        if platform[i][rightPointer] == '#': # if it founds an immovable rock, update leftPointer
          leftPointer = rightPointer
          continue
    return platform

def tiltNorth(platform):
  platform = rotateCCW90(platform)
  platform = tiltWest(platform)
  platform = rotateCCW90(rotateCCW90(rotateCCW90(platform))) # lmao i can make a rotateCW90 function but this is just too funny
  return platform

def tiltEast(platform):
  platform = rotateCCW90(rotateCCW90(platform))
  platform = tiltWest(platform)
  platform = rotateCCW90(rotateCCW90(platform))
  return platform

def tiltSouth(platform):
  platform = rotateCCW90(rotateCCW90(rotateCCW90(platform)))
  platform = tiltWest(platform)
  platform = rotateCCW90(platform)
  return platform

def totalTiltCycles(platform, multiplier):
  # finding the cycle
  weightCycle = []
  seenWeight = set()
  cycleStart = None
  for i in range(multiplier): # the cycle
    platform = tiltEast(tiltSouth(tiltWest(tiltNorth(platform))))
    weight = countWeightNorth(platform)

    # if weight in seenWeight: # if we see a weight again ...
    #   cycleStart = weightCycle.index(weight) # then the cycle starts at the first occurence of weight
    #   break # then finish finding the cycle

    # if we haven't seen one, just continue
    weightCycle.append(weight)
    seenWeight.add(weight)

  # after we exit the loop
  if cycleStart is not None: # if we find a cycle
    cycle = weightCycle[cycleStart:] # this is the cycle we see
    cycleLength = len(weightCycle) - cycleStart

    # we want to know what the value'd be at index = multiplier - 1 (at the end)
    _, remainder = divmod(((multiplier - 1) - cycleStart), cycleLength)
    return weightCycle[remainder] # return the one at remainder

  # if we don't find a cycle then
  print(weightCycle)
  return weightCycle[multiplier - 1]

# countWeightNorth is still the same

# solution
# inputFilename = 'advent-of-code-day14-input'
inputFilename = 'advent-of-code-day14-testcase'
multiplier = 111
platform = cleanInput(inputFilename)
totalTiltCycles(platform, multiplier)

In [ ]:
# first iteration
import functools
import sys
# lmao i think this isn't possible even in small matrices like these ok now think from first principle
sys.setrecursionlimit(10**5)

@functools.cache # lmao memoize!!!! (this feels so dumb hahahahaha)
def tiltNorth(platform):
  tempPlatform = [list(row) for row in platform]
  for i in range(len(platform)-1):
    for j, pair in enumerate(zip(platform[i], platform[i + 1])):
      north, south = pair
      if north == '.' and south == 'O':
        # update if it falls over
        tempPlatform[i][j] = 'O' # the one at north
        tempPlatform[i + 1][j] = '.' # the one at south
  tempPlatform = tuple(tuple(row) for row in tempPlatform)

  # too deep too much man lmao
  if platform == tempPlatform: # base case
    return platform
  return tiltNorth(platform) # recursive
  # return (platform, tempPlatform)

tiltNorth(platform)

((('O', '.', '.', '.', '.', '#', '.', '.', '.', '.'),
  ('O', '.', 'O', 'O', '#', '.', '.', '.', '.', '#'),
  ('.', '.', '.', '.', '.', '#', '#', '.', '.', '.'),
  ('O', 'O', '.', '#', 'O', '.', '.', '.', '.', 'O'),
  ('.', 'O', '.', '.', '.', '.', '.', 'O', '#', '.'),
  ('O', '.', '#', '.', '.', 'O', '.', '#', '.', '#'),
  ('.', '.', 'O', '.', '.', '#', 'O', '.', '.', 'O'),
  ('.', '.', '.', '.', '.', '.', '.', 'O', '.', '.'),
  ('#', '.', '.', '.', '.', '#', '#', '#', '.', '.'),
  ('#', 'O', 'O', '.', '.', '#', '.', '.', '.', '.')),
 (('O', '.', 'O', 'O', '.', '#', '.', '.', '.', '.'),
  ('O', '.', '.', '.', '#', '.', '.', '.', '.', '#'),
  ('O', 'O', '.', '.', 'O', '#', '#', '.', '.', 'O'),
  ('.', '.', '.', '#', '.', '.', '.', 'O', '.', '.'),
  ('O', 'O', '.', '.', '.', 'O', '.', '.', '#', '.'),
  ('.', '.', '#', '.', '.', '.', 'O', '#', '.', '#'),
  ('.', '.', 'O', '.', '.', '#', '.', 'O', '.', 'O'),
  ('.', '.', '.', '.', '.', '.', '.', '.', '.', '.'),
  ('#', 'O', 'O', '.', '.',